In [1]:
import requests
import json
import pandas as pd

In [2]:
url = "https://doi.crossref.org/getPrefixPublisher/**"
data = requests.get(url).json()

In [21]:
res = []
for elt in data:
    for doi_prefix in list(set(elt['prefixes'])):
        publisher = elt['name']
        res.append({'doi_prefix': doi_prefix.strip(), 'publisher': publisher.strip()})
df = pd.DataFrame(res)
df.to_csv('publisher_doi.csv', index=False)

In [79]:
df_2013 = pd.read_csv('https://gist.githubusercontent.com/TomDemeranville/8699224/raw/9a5ca0eed28a24bf83e5fc96962d9d008218949f/doi-prefix-publishers.csv')
df_2013['doi_prefix'] = df_2013.prefix.astype(str)
#del df_2013['prefix']
del df_2013['journals']
del df_2013['dois']

In [23]:
df_comparaison = pd.merge(df, df_2013, on='doi_prefix', how='outer', indicator=True)
df_comparaison.columns = ['doi_prefix', 'publisher_2022', 'publisher_2013', '_merge']

In [48]:
df_new = df_comparaison[df_comparaison._merge == 'left_only']
del df_new['_merge']

In [49]:
df_removed = df_comparaison[df_comparaison._merge == 'right_only']
del df_removed['_merge']

In [50]:
df_changed = df_comparaison[(df_comparaison._merge == 'both') & (df_comparaison.publisher_2022 != df_comparaison.publisher_2013) ]
del df_changed['_merge']

In [59]:
df_unchanged = df_comparaison[(df_comparaison._merge == 'both') & (df_comparaison.publisher_2022 == df_comparaison.publisher_2013) ]
del df_unchanged['_merge']

In [51]:
df_new.to_csv('new_prefix_2013_2022.csv', index=False)
df_removed.to_csv('removed_prefix_2013_2022.csv', index=False)
df_changed.to_csv('changed_prefix_2013_2022.csv', index=False)

In [52]:
len(df_new)

19344

In [53]:
len(df)

22416

In [54]:
len(df_2013)

3417

In [55]:
len(df_removed)

312

In [56]:
len(df_changed)

1322

In [62]:
3417 + 19344

22761

In [61]:
len(df_comparaison)

22761

In [63]:
df_comparaison._merge.value_counts()

left_only     19344
both           3105
right_only      312
Name: _merge, dtype: int64

In [64]:
3417 - 312 + 19344

22449

In [65]:
len(df)

22416

In [66]:
19344 + 3105

22449

In [69]:
set(df_comparaison[df_comparaison._merge != 'right_only'].doi_prefix) - set(df.doi_prefix)

set()

In [71]:
len(df_comparaison[df_comparaison._merge != 'right_only'].doi_prefix)

22449

In [72]:
len((df.doi_prefix))

22416

In [73]:
len(set(df_comparaison[df_comparaison._merge != 'right_only'].doi_prefix))

22416

In [76]:
len(df_2013.doi_prefix.unique())

3381

In [77]:
df_2013.doi_prefix.value_counts()

10.1159     2
10.1298     2
10.1271     2
10.1161     2
10.1273     2
           ..
10.565      1
10.1641     1
10.3896     1
10.3813     1
10.11634    1
Name: doi_prefix, Length: 3381, dtype: int64

In [80]:
df_2013[df_2013.doi_prefix=='10.1159']

,prefix,publisher,doi_prefix
756,10.1159,Department of International Relations,10.1159
2396,10.1159,S. Karger AG,10.1159
